In [1]:
# -*- coding: utf-8
# Abraji (https://www.abraji.org.br)
# Reinaldo Chaves (reinaldo@abraji.org.br)
# Programa 2 para pegar movimentações de processos na busca do 1º Grau 
# Com o arquivo gerado no programa 1 entra em cada página de processo
# e capta informações iniciais do processo
#

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import urllib3; urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import time
import pandas as pd

In [2]:
df_processos = pd.read_excel('resultados/processos_tjsp_22ago.xlsx', Sheet="Sheet1")

In [3]:
df_processos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5199 entries, 0 to 5198
Data columns (total 7 columns):
process     5199 non-null object
link        5199 non-null object
type        5199 non-null object
label       5199 non-null object
name        5199 non-null object
date        5199 non-null datetime64[ns]
location    5199 non-null object
dtypes: datetime64[ns](1), object(6)
memory usage: 284.4+ KB


In [4]:
processos = []

In [5]:
for num, row in df_processos.iterrows():
    link = row['link']
    num_processo = row['process']
    tipo = row['type']
    parte_especie = row['label']
    nome_parte = row['name']
    data = row['date']
    foro = row['location']
    
    time.sleep(1)
    
    r = requests.get(link, verify=False)
    
    soup = bs(r.content, 'lxml')
    
    assunto = soup.select_one('td:has(>.labelClass:contains("Assunto:")) + td').text.strip() if soup.select_one('td:has(>.labelClass:contains("Assunto:")) + td') is not None else 'N/A'
    
    mais_detalhes = soup.select_one('td:has(>.labelClass:contains("Outros assuntos:")) + td').text.strip() if soup.select_one('td:has(>.labelClass:contains("Outros assuntos:")) + td') is not None else 'N/A'
    
    data_distribuicao = soup.select_one('td:has(>.labelClass:contains("Distribuição:")) + td').text.strip() if soup.select_one('td:has(>.labelClass:contains("Distribuição:")) + td') is not None else 'N/A'
    
    magistrado = soup.select_one('td:has(>.labelClass:contains("Juiz:")) + td').text.strip() if soup.select_one('td:has(>.labelClass:contains("Juiz:")) + td') is not None else 'N/A'
    
    valor_acao = soup.select_one('td:has(>.labelClass:contains("Valor da ação:")) + td').text.strip() if soup.select_one('td:has(>.labelClass:contains("Valor da ação:")) + td') is not None else 'N/A'
    
    
    processos.append({'num_processo': num_processo,
                      'link': link,
                      'tipo': tipo,
                      'parte_especie': parte_especie,
                      'nome_parte': nome_parte,
                      'data': data,
                      'foro': foro,
                      'assunto': assunto,
                      'mais_detalhes': mais_detalhes,
                      'data_distribuicao': data_distribuicao,
                      'magistrado': magistrado,
                      'valor_acao': valor_acao
                     }
                      )    
    

In [6]:
df_processos_mais = pd.DataFrame(processos, columns = ['num_processo', 
                                                       'link', 
                                                       'tipo',
                                                       'parte_especie',
                                                       'nome_parte',
                                                       'data',
                                                       'foro', 
                                                       'assunto', 
                                                       'mais_detalhes', 
                                                       'data_distribuicao',
                                                       'magistrado', 
                                                       'valor_acao'])

In [7]:
df_processos_mais.reset_index().head(10)

,index,num_processo,link,tipo,parte_especie,nome_parte,data,foro,assunto,mais_detalhes,data_distribuicao,magistrado,valor_acao
0,0,1614864-57.2018.8.26.0224,https://esaj.tjsp.jus.br/cpopg/show.do?process...,DIREITO TRIBUTÁRIO,Exectdo,Troad Editora e Jornal Expresso Metropolitano ...,2018-08-30,Vara da Fazenda Pública,DIREITO TRIBUTÁRIO,N/A,30/08/2018 às 09:04 - Livre,Rafael Tocantins Maltez,"R$ 9.453,66"
1,1,1050808-03.2018.8.26.0053,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Base de Cálculo,Reqte,Jornal Gazeta de Sao Paulo - Epp,2018-11-10,Vara de Fazenda Pública,Base de Cálculo,Anulação de Débito Fiscal,15/10/2018 às 13:29 - Livre,Randolfo Ferraz de Campos,"R$ 44.200,76"
2,2,1045411-60.2018.8.26.0053,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Anulação de Débito Fiscal,Reqte,Jornal Gazeta de São Paulo EPP,2018-09-13,Vara de Fazenda Pública,Anulação de Débito Fiscal,Base de Cálculo,13/09/2018 às 15:05 - Livre,Otavio Tioiti Tokuda,"R$ 159.006,76"
3,3,1000221-74.2018.8.26.0053,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Anulação de Débito Fiscal,Reqte,Jornal Gazeta de São Paulo Ltda - Epp,2018-05-01,Vara de Fazenda Pública,Anulação de Débito Fiscal,N/A,09/01/2018 às 13:24 - Livre,LUIS MANUEL FONSECA PIRES,"R$ 45.701,54"
4,4,1042815-40.2017.8.26.0053,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Antecipação de Tutela / Tutela Específica,Reqte,Jornal Gazeta de São Paulo - Epp,2017-09-13,Vara do Juizado Especial da Fazenda Pública,Antecipação de Tutela / Tutela Específica,Liminar,18/09/2017 às 16:44 - Livre,ENIO JOSE HAUFFE,"R$ 10.045,46"
5,5,1079553-12.2019.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Indenização por Dano Moral,Reqdo,Editora Agora Parana – Jornal Agora Parana,2019-08-15,Vara Cível,Indenização por Dano Moral,Direito de Imagem,15/08/2019 às 17:21 - Livre,Adriana Sachsida Garcia,"R$ 120.000,00"
6,6,1076696-90.2019.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Duplicata,Reqte,Jornal Gazeta de Sao Paulo Ltda,2019-08-08,Vara Cível,Duplicata,N/A,08/08/2019 às 12:08 - Livre,Gisele Valle Monteiro da Rocha,"R$ 6.516,28"
7,7,1056775-48.2019.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Levantamento de Valor,Exeqte,Jornal A Gazeta Ltda.,2019-06-13,VARA EMPRESARIAL E CONFLITOS DE ARBITRAGEM,Levantamento de Valor,N/A,23/07/2019 às 12:47 - Livre,Renata Mota Maciel,"R$ 1.055.268,03"
8,8,1054881-37.2019.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Marca,Reqdo,A Tribuna de Santos Jornal e Editora Ltda.,2019-07-06,VARA EMPRESARIAL E CONFLITOS DE ARBITRAGEM,Marca,N/A,07/06/2019 às 17:47 - Livre,LUIS FELIPE FERRARI BEDENDI,"R$ 50.000,00"
9,9,1052757-81.2019.8.26.0100,https://esaj.tjsp.jus.br/cpopg/show.do?process...,Indenização por Dano Moral,Reqdo,Jornal do Síndico,2019-03-06,Vara Cível,Indenização por Dano Moral,N/A,03/06/2019 às 14:16 - Livre,Thania Pereira Teixeira De Carvalho Cardin,"R$ 5.000,00"


In [8]:
df_processos_mais.to_excel('resultados/processos_tjsp_22ago_segunda_fase.xlsx',sheet_name='Sheet1')

In [9]:
df_processos_mais.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5199 entries, 0 to 5198
Data columns (total 12 columns):
num_processo         5199 non-null object
link                 5199 non-null object
tipo                 5199 non-null object
parte_especie        5199 non-null object
nome_parte           5199 non-null object
data                 5199 non-null datetime64[ns]
foro                 5199 non-null object
assunto              5199 non-null object
mais_detalhes        5199 non-null object
data_distribuicao    5199 non-null object
magistrado           5199 non-null object
valor_acao           5199 non-null object
dtypes: datetime64[ns](1), object(11)
memory usage: 487.5+ KB
